In [1]:
!pip install flask flask-ngrok diffusers torch
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 19.7 MB/s eta 0:00:00


In [4]:
!ngrok authtoken 2luZw1XEYVhmfscnSt5dNinhcLq_5MGgTgdX1WrhLqd43f5Up

from pyngrok import ngrok

# Open a tunnel for Flask app
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://a9e7-34-168-64-194.ngrok-free.app" -> "http://localhost:5000"


In [5]:
#colab.py

from flask import Flask, request, jsonify, send_from_directory
from diffusers import AutoencoderKL, DiffusionPipeline
import torch
import os
from PIL import Image
from flask_ngrok import run_with_ngrok
import logging

# Set up logging
logging.basicConfig(level=logging.DEBUG)

app = Flask(__name__)
run_with_ngrok(app)  # Starts ngrok when the app runs

# Define the temporary directory for storing images
TEMP_DIR = '/tmp'

@app.route('/process', methods=['POST'])
def process():
    try:
        # Get images and prompt from request
        images = request.files.getlist('images')
        prompt = request.form.get('prompt', '')

        # Validate prompt
        if not prompt:
            return jsonify({'error': 'Prompt is required'}), 400

        # Ensure the temporary directory exists
        if not os.path.exists(TEMP_DIR):
            os.makedirs(TEMP_DIR)

        # Save images to the temporary directory
        image_paths = []
        for image in images:
            if image:
                path = os.path.join(TEMP_DIR, image.filename)
                image.save(path)
                image_paths.append(path)
            else:
                logging.error("Image file is missing.")
                return jsonify({'error': 'Image file is missing'}), 400

        # Load the AutoencoderKL model
        try:
            vae = AutoencoderKL.from_pretrained(
                "madebyollin/sdxl-vae-fp16-fix",
                torch_dtype=torch.float16
            )
        except Exception as e:
            logging.error(f"Error loading VAE model: {e}")
            return jsonify({'error': 'Error loading VAE model'}), 500

        # Load the DiffusionPipeline model
        try:
            pipe = DiffusionPipeline.from_pretrained(
                "stabilityai/stable-diffusion-xl-base-1.0",
                vae=vae,
                torch_dtype=torch.float16,
                variant="fp16",
                use_safetensors=True
            )
        except Exception as e:
            logging.error(f"Error loading DiffusionPipeline model: {e}")
            return jsonify({'error': 'Error loading DiffusionPipeline model'}), 500

        # Set the target device to CUDA (GPU)
        pipe.to("cuda")

        # Generate the image
        try:
            image = pipe(prompt=prompt, num_inference_steps=10, num_images_per_prompt=1)
            generated_image = image.images[0]
        except Exception as e:
            logging.error(f"Error generating image: {e}")
            return jsonify({'error': 'Error generating image'}), 500

        # Save the generated image
        generated_image_filename = 'generated_image.png'
        output_path = os.path.join(TEMP_DIR, generated_image_filename)
        generated_image.save(output_path)

        # Check if the file exists
        if not os.path.isfile(output_path):
            logging.error(f"Generated image file not found: {output_path}")
            return jsonify({'error': 'Generated image file not found'}), 500

        # Return the URL where the generated image can be accessed
        return jsonify({'image_url': f'/generated_image/{generated_image_filename}'})

    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return jsonify({'error': 'An unexpected error occurred'}), 500

@app.route('/generated_image/<filename>')
def serve_image(filename):
    """Serve the generated image from the temporary directory."""
    return send_from_directory(TEMP_DIR, filename)

if __name__ == '__main__':
    app.run()  # Do not specify port here


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://a9e7-34-168-64-194.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [11/Sep/2024 11:17:17] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2024 11:17:18] "GET /generated_image/generated_image.png HTTP/1.1" 200 -


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [11/Sep/2024 11:19:55] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2024 11:19:57] "GET /generated_image/generated_image.png HTTP/1.1" 200 -
